In [1]:
import pandas as pd
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from gensim.models import KeyedVectors
import re
import gensim

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aryuska/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv("./description-final.csv")


In [3]:
df=df.dropna()

In [4]:

def _removeNonAscii(s):
    return "".join(i.encode('ascii', 'ignore').decode() for i in s )

def make_lower_case(text):
    return text.lower()

def remove_stop_words(text):
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text

def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

def remove_punctuation(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    text = " ".join(text)
    return text

In [5]:
df['cleaned'] = df['description'].apply(_removeNonAscii)

df['cleaned'] = df.cleaned.apply(func = make_lower_case)
df['cleaned'] = df.cleaned.apply(func = remove_stop_words)
df['cleaned'] = df.cleaned.apply(func=remove_punctuation)
df['cleaned'] = df.cleaned.apply(func=remove_html)

In [6]:
corpus = []
for words in df['cleaned']:
    corpus.append(words.split())
print(corpus[0])
print(corpus[2])

['documentary', 'pro', 'skateboarder', 'danny', 'way', 's', 'tough', 'childhood', 'contributions', 'sport', 'including', 'footage', 'jump', 'great', 'wall', 'china']
['show', 'business', 'family', 'leaves', 'great', 'white', 'way', 'heads', 'farm', 'new', 'england', 'results', 'difficulties', 'accepted', 'community']


In [7]:
model = gensim.models.Word2Vec(corpus, min_count=1)

In [8]:
df_rat = pd.read_csv("./ml-25m/ratings.csv")

In [9]:
df_rat_mov = pd.merge(df_rat, df, on="movieId", how="inner")

In [10]:
df_rat_mov_3 = df_rat_mov[df_rat_mov['userId'] == 3]
df_rat_mov_3

,userId,movieId,rating,timestamp,title,description,cleaned
0,3,101864,4.0,1439474698,Oblivion,A veteran assigned to extract Earth's remainin...,veteran assigned extract earth s remaining res...
5339,3,102125,4.0,1439474937,Iron Man Three,When Tony Stark's world is torn apart by a for...,tony stark s world torn apart formidable terro...
13604,3,102407,4.0,1566090339,The Great Gatsby,"A writer and wall street trader, Nick, finds h...",writer wall street trader nick finds drawn pas...
18250,3,102445,4.0,1439473797,Star Trek Into Darkness,After the crew of the Enterprise find an unsto...,crew enterprise find unstoppable force terror ...
25713,3,102686,3.0,1566091223,The Hangover Part III,When one of their own is kidnapped by an angry...,one kidnapped angry gangster wolf pack must tr...
...,...,...,...,...,...,...,...
581873,3,176371,4.5,1566090291,Blade Runner 2049,Young Blade Runner K's discovery of a long-bur...,young blade runner k s discovery long buried s...
586992,3,179819,3.5,1566090591,Star Wars: Episode VIII - The Last Jedi,The Star Wars saga continues as new heroes and...,star wars saga continues new heroes galactic l...
590299,3,182823,3.0,1566089510,Bright,A detective must work with an Orc to find a po...,detective must work orc find powerful wand evi...
591226,3,187541,3.5,1566090693,Incredibles 2,The Incredibles family takes on a new mission ...,incredibles family takes new mission involves ...


In [11]:
df_rat_mov_3 = df_rat_mov_3.sort_values(by='rating', ascending=False)
df_rat_mov_3 = df_rat_mov_3[df_rat_mov_3['rating'] >= 3.5]

In [12]:
df_rat_mov_3

,userId,movieId,rating,timestamp,title,description,cleaned
507784,3,136449,5.0,1484753762,Kôkaku kidôtai 2.0,A hacker known as the Puppet Master is hunted ...,hacker known puppet master hunted female cybor...
264719,3,112556,5.0,1439473517,Gone Girl,With his wife's disappearance having become th...,wife s disappearance become focus intense medi...
253540,3,112552,5.0,1439472513,Whiplash,A promising young drummer enrolls at a cut-thr...,promising young drummer enrolls cut throat mus...
327627,3,115713,4.5,1439474672,Ex Machina,A young programmer is selected to participate ...,young programmer selected participate ground b...
68805,3,104243,4.5,1439474932,Riddick,"Left for dead on a sun-scorched planet, Riddic...",left dead sun scorched planet riddick finds al...
...,...,...,...,...,...,...,...
400477,3,122892,3.5,1484755456,Avengers: Age of Ultron,When Tony Stark and Bruce Banner try to jump-s...,tony stark bruce banner try jump start dormant...
408198,3,122896,3.5,1566091420,Pirates of the Caribbean: Dead Men Tell No Tales,Captain Jack Sparrow is pursued by old rival C...,captain jack sparrow pursued old rival captain...
110606,3,106489,3.5,1566091626,The Hobbit: The Desolation of Smaug,"The dwarves, along with Bilbo Baggins and Gand...",dwarves along bilbo baggins gandalf grey conti...
586992,3,179819,3.5,1566090591,Star Wars: Episode VIII - The Last Jedi,The Star Wars saga continues as new heroes and...,star wars saga continues new heroes galactic l...


In [13]:
user_pref = ""
for i,data in df_rat_mov_3.iterrows():
    user_pref += " " + data['cleaned']

In [14]:
corpus_user_pref = []
corpus_user_pref.append(user_pref.split())
    

In [15]:
corpus_user_pref

[['hacker',
  'known',
  'puppet',
  'master',
  'hunted',
  'female',
  'cyborg',
  'cop',
  'partner',
  'film',
  'revised',
  'version',
  'kkaku',
  'kidtai',
  '1995',
  'wife',
  's',
  'disappearance',
  'become',
  'focus',
  'intense',
  'media',
  'circus',
  'man',
  'sees',
  'spotlight',
  'turned',
  'suspected',
  'may',
  'innocent',
  'promising',
  'young',
  'drummer',
  'enrolls',
  'cut',
  'throat',
  'music',
  'conservatory',
  'dreams',
  'greatness',
  'mentored',
  'instructor',
  'stop',
  'nothing',
  'realize',
  'student',
  's',
  'potential',
  'young',
  'programmer',
  'selected',
  'participate',
  'ground',
  'breaking',
  'experiment',
  'synthetic',
  'intelligence',
  'evaluating',
  'human',
  'qualities',
  'highly',
  'advanced',
  'humanoid',
  'a',
  'i',
  'left',
  'dead',
  'sun',
  'scorched',
  'planet',
  'riddick',
  'finds',
  'alien',
  'race',
  'predators',
  'activating',
  'emergency',
  'beacon',
  'alerts',
  'two',
  'ships'

In [16]:
df_recommend = df
df_recommend

,title,description,movieId,cleaned
0,Waiting for Lightning,A documentary on pro skateboarder Danny Way's ...,99513,documentary pro skateboarder danny way s tough...
1,Yesterday's Enemy,"In World War II, during the Japanese invasion ...",99515,world war ii japanese invasion burma lost remn...
2,Young People,A show-business family leaves the Great White ...,99517,show business family leaves great white way he...
3,Henry,Henry is a cynical man who owns a musical inst...,99519,henry cynical man owns musical instruments sto...
4,L'incruste,A small crook manages to crash at a musician's...,99521,small crook manages crash musician s apartment...
...,...,...,...,...
43322,Wij,Eight teens - four guys and four girls - are t...,209157,eight teens four guys four girls best friends ...
43323,Janela da Alma,Nineteen people with differing degrees of visu...,209159,nineteen people differing degrees visual impai...
43324,Rossz versek,"33-years old Tamás Merthner is heartbroken, af...",209163,33 years old tams merthner heartbroken girlfri...
43325,A Girl Thing,An NYC shrink sees four different women with d...,209169,nyc shrink sees four different women different...


In [17]:
def get_similarity_score(text):
    return model.wv.n_similarity(corpus_user_pref[0], text.split())

In [19]:
df_recommend['score'] = df.cleaned.apply(get_similarity_score)

In [25]:
df_recommend = df_recommend.sort_values(by='score', ascending=False)
df_recommend

,title,description,movieId,cleaned,score
7718,Zwischen Welten,Jesper is a soldier in the German army. Althou...,127254,jesper soldier german army although brother ki...,0.995100
6109,The Adventures of Tartu,1940. Captain Terence Stevenson with the Briti...,123219,1940 captain terence stevenson british army pa...,0.994002
11848,Tom and Jerry: Spy Quest,Two groups of classic cartoon characters come ...,137383,two groups classic cartoon characters come tog...,0.993831
15465,Letters from the Big Man,"Sarah Smith, an artist and government hydrolog...",145741,sarah smith artist government hydrologist sets...,0.993519
25673,Sengoku jieitai,A squadron of Japanese Self-Defense Force sold...,168938,squadron japanese self defense force soldiers ...,0.993269
...,...,...,...,...,...
3646,O Amor das Três Romãs,A story of love and spells.,116102,story love spells,0.525065
577,Going to Pieces: The Rise and Fall of the Slas...,A documentary on the history of the slasher film.,102111,documentary history slasher film,0.511730
39761,Pinocchio,The enchanted story of Pinocchio.,199540,enchanted story pinocchio,0.502054
35202,Confessions of a Porn Addict,a love story...,189247,love story,0.490911


In [24]:
df_rat_mov_grouped = df_rat_mov.groupby('movieId')['rating'].mean()

In [26]:
df_recommend = pd.merge(df_recommend, df_rat_mov_grouped, on="movieId",how='left')

In [30]:
import numpy as np
def dcg_at_k(r, k):
    """
    Calculate DCG (Discounted Cumulative Gain) at k for a list of item ratings.
    
    Args:
    - r: list of item ratings
    - k: integer, number of items to consider
    
    Returns:
    - DCG@k: float, Discounted Cumulative Gain at k
    """
    r = np.asfarray(r)[:k]
    if r.size:
        return np.sum(np.subtract(np.power(2, r), 1) / np.log2(np.arange(2, r.size + 2)))
    return 0.0

def ndcg_at_k(r, k):
    """
    Calculate NDCG (Normalized Discounted Cumulative Gain) at k for a list of item ratings.
    
    Args:
    - r: list of item ratings
    - k: integer, number of items to consider
    
    Returns:
    - NDCG@k: float, Normalized Discounted Cumulative Gain at k
    """
    dcg_max = dcg_at_k(sorted(r, reverse=True), k)
    if not dcg_max:
        return 0.0
    return dcg_at_k(r, k) / dcg_max

In [32]:
df_recommend = df_recommend.dropna()

In [33]:
# ndcg cbf 
df_recommend = df_recommend.sort_values('score', ascending=False)
item_ratings = df_recommend['rating'] # item ratings in the ranked order
k = 5
ndcg = ndcg_at_k(item_ratings, k)

print('NDCG@{}: {:.2f}'.format(k, ndcg))

NDCG@5: 0.25


In [36]:
train_ratings = df_rat.sample(frac=0.8, random_state=1)
test_ratings = df_rat.drop(train_ratings.index)

In [38]:
user_means = train_ratings.groupby('userId')['rating'].mean()

In [39]:
train_ratings['Rating Adjusted'] = train_ratings['rating'] - train_ratings['userId'].map(user_means)

In [40]:
movie_weights = train_ratings.groupby('movieId')['Rating Adjusted'].sum() / train_ratings.groupby('movieId')['Rating Adjusted'].count()

In [43]:
df_recommend['userId'] = 3

In [44]:
df_recommend['Predicted Rating'] = df_recommend.apply(lambda row: user_means.get(row['userId'], 0) + movie_weights.get(row['movieId'], 0), axis=1)

In [49]:
def binRating(rating):
    if rating >= 3.5:
        return 1
    else:
        return 0

In [50]:
df_recommend['BIN AVG Rating'] = df_recommend['rating'].apply(binRating)
df_recommend['BIN Predicted Rating'] = df_recommend['Predicted Rating'].apply(binRating)

In [51]:
df_recommend.head()

,title,description,movieId,cleaned,score,rating,userId,Predicted Rating,BIN AVG Rating,BIN Predicted Rating
0,Zwischen Welten,Jesper is a soldier in the German army. Althou...,127254,jesper soldier german army although brother ki...,0.995100,2.750000,3,3.137527,0,0
1,The Adventures of Tartu,1940. Captain Terence Stevenson with the Briti...,123219,1940 captain terence stevenson british army pa...,0.994002,3.666667,3,3.631250,1,1
2,Tom and Jerry: Spy Quest,Two groups of classic cartoon characters come ...,137383,two groups classic cartoon characters come tog...,0.993831,1.500000,3,2.435314,0,0
3,Letters from the Big Man,"Sarah Smith, an artist and government hydrolog...",145741,sarah smith artist government hydrologist sets...,0.993519,3.000000,3,3.855923,0,1
5,Karas: The Prophecy,In a Japanese city haunted by nature spirits a...,157777,japanese city haunted nature spirits demons se...,0.992973,4.000000,3,4.348089,1,1


In [46]:
def precision_recall_at_k(recommended_items, relevant_items, k):
    """
    Calculate precision@k and recall@k for a recommendation system.
    
    Args:
    - recommended_items: list of recommended items
    - relevant_items: list of relevant items
    - k: integer, number of items to consider
    
    Returns:
    - precision@k: float, precision at k
    - recall@k: float, recall at k
    """
    
    recommended_items = recommended_items[:k]
    
    # Calculate precision@k
    precision = len(set(recommended_items).intersection(set(relevant_items))) / float(k)
    
    # Calculate recall@k
    recall = len(set(recommended_items).intersection(set(relevant_items))) / float(len(relevant_items))
    
    return precision, recall

In [53]:
df_recommend = df_recommend.sort_values('score',ascending=False)
k = 10
precision, recall = precision_recall_at_k(df_recommend['BIN Predicted Rating'], df_recommend['BIN AVG Rating'], k)

print('Precision@{}: {:.2f}'.format(k, precision))
print('Recall@{}: {:.2f}'.format(k, recall))

Precision@10: 0.20
Recall@10: 0.00


/var/folders/wp/36kdrjm51z92ythp0vp8xp5m0000gn/T/ipykernel_1231/2063049648.py:15: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  recommended_items = recommended_items[:k]


In [45]:
df_recommend

,title,description,movieId,cleaned,score,rating,userId,Predicted Rating
0,Zwischen Welten,Jesper is a soldier in the German army. Althou...,127254,jesper soldier german army although brother ki...,0.995100,2.750000,3,3.137527
1,The Adventures of Tartu,1940. Captain Terence Stevenson with the Briti...,123219,1940 captain terence stevenson british army pa...,0.994002,3.666667,3,3.631250
2,Tom and Jerry: Spy Quest,Two groups of classic cartoon characters come ...,137383,two groups classic cartoon characters come tog...,0.993831,1.500000,3,2.435314
3,Letters from the Big Man,"Sarah Smith, an artist and government hydrolog...",145741,sarah smith artist government hydrologist sets...,0.993519,3.000000,3,3.855923
5,Karas: The Prophecy,In a Japanese city haunted by nature spirits a...,157777,japanese city haunted nature spirits demons se...,0.992973,4.000000,3,4.348089
...,...,...,...,...,...,...,...,...
43320,Charles Darwin and the Tree of Life,A documentary about evolution.,165059,documentary evolution,0.530369,4.166667,3,4.756971
43321,O Amor das Três Romãs,A story of love and spells.,116102,story love spells,0.525065,3.000000,3,3.432304
43322,Going to Pieces: The Rise and Fall of the Slas...,A documentary on the history of the slasher film.,102111,documentary history slasher film,0.511730,3.361111,3,3.948881
43323,Pinocchio,The enchanted story of Pinocchio.,199540,enchanted story pinocchio,0.502054,3.500000,3,4.723429


In [141]:
df_rat_mov_3

,userId,movieId,rating,timestamp,title,description,cleaned
507784,3,136449,5.0,1484753762,Kôkaku kidôtai 2.0,A hacker known as the Puppet Master is hunted ...,hacker known puppet master hunted female cybor...
264719,3,112556,5.0,1439473517,Gone Girl,With his wife's disappearance having become th...,wife s disappearance become focus intense medi...
253540,3,112552,5.0,1439472513,Whiplash,A promising young drummer enrolls at a cut-thr...,promising young drummer enrolls cut throat mus...
327627,3,115713,4.5,1439474672,Ex Machina,A young programmer is selected to participate ...,young programmer selected participate ground b...
68805,3,104243,4.5,1439474932,Riddick,"Left for dead on a sun-scorched planet, Riddic...",left dead sun scorched planet riddick finds al...
581873,3,176371,4.5,1566090291,Blade Runner 2049,Young Blade Runner K's discovery of a long-bur...,young blade runner k s discovery long buried s...
118851,3,106782,4.5,1439473659,The Wolf of Wall Street,"Based on the true story of Jordan Belfort, fro...",based true story jordan belfort rise wealthy s...
594096,3,187593,4.5,1566090414,Deadpool 2,Foul-mouthed mutant mercenary Wade Wilson (a.k...,foul mouthed mutant mercenary wade wilson a k ...


In [151]:
df_recommend = df_recommend[:10]

In [155]:
df_recommend

,title,description,movieId,cleaned,score
32963,Who Is Clark Rockefeller?,"Called ""the longest running con in FBI history...",184487,called the longest running con fbi history chr...,0.992058
4389,Fortune and Men's Eyes,"The young, naive Smitty is sent to prison for ...",118232,young naive smitty sent prison six months cath...,0.991181
25465,Les Patterson Saves the World,Fat middle aged 24/7 drunkard Les Patterson (B...,168482,fat middle aged 24 7 drunkard les patterson ba...,0.991004
33231,Le plaisir de chanter,A banker has died. He trafficked in nuclear ma...,185043,banker died trafficked nuclear material french...,0.990824
16531,'Z' fung bou,This is a story about the biggest financial fr...,148270,story biggest financial fraud attempted hong k...,0.990312
39591,Soldados de Salamina,When the professor and writer Lola Sánchez is ...,199131,professor writer lola snchez assigned write co...,0.989737
13657,2033,"The first installment of a planned trilogy, di...",141652,first installment planned trilogy director fra...,0.989707
5406,Sons o' Guns,Broadway star Jimmy Canfield stars in a patrio...,121642,broadway star jimmy canfield stars patriotic s...,0.989305
7479,Kosmicheskiy reys: Fantasticheskaya novella,"In 1946, the Soviet space program is undergoin...",126436,1946 soviet space program undergoing turmoil p...,0.989061
17322,Proof of the Devil,A string of murders ended in the capture of an...,150190,string murders ended capture 18 yo boy named j...,0.988768
